<a href="https://colab.research.google.com/github/Trading-com-Dados/redes-sociais/blob/main/20220419_MythBusters_formato_candles_indica_futuro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300">

---
# **MythBusters do mercado financeiro:**
## "*O formato de um candle revela os próximos movimentos do mercado*"
---

# **1. Bibliotecas utilizadas**

In [59]:
!pip install yfinance
!pip install python-binance
!pip install TA-Lib
!pip install pandas_ta

     |████████████████████████████████| 65 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 10.1 MB/s 
     |████████████████████████████████| 45 kB 3.5 MB/s 
     |████████████████████████████████| 288 kB 56.9 MB/s 
     |████████████████████████████████| 112 kB 46.9 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 61.6 MB/s 
     |████████████████████████████████| 144 kB 74.6 MB/s 
  Using cached TA-Lib-0.4.24.tar.gz (269 kB)
  ERROR: Failed building wheel for TA-Lib
  Running setup.py clean for TA-Lib
Failed to build TA-Lib
    Running setup.py install for TA-Lib ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-m7xc646z/ta-lib_9763deb98b76472b9f4ea4a188ef333c/setup.py'"'"'; __file__='"'"'/tmp/pip-install-m7xc646z/ta-lib_9763deb98b76472b9f4ea4a188ef333c/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', o

In [89]:
import numpy as np
import pandas as pd
from datetime import datetime

import yfinance as yf
import pandas_ta as ta
from binance.client import Client

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# **2. Obter cotações ativos**

In [31]:
tickers_cotacoes = ['^BVSP']

In [32]:
cotacoes_ohlc = yf.download(tickers_cotacoes, start = "2021-01-01", end = "2022-01-01",interval = "1D")
cotacoes_ohlc

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,119024.0,120354.0,118062.0,118558.0,118558.0,8741400
2021-01-05,118835.0,119790.0,116756.0,119223.0,119223.0,9257100
2021-01-06,119377.0,120924.0,118917.0,119851.0,119851.0,11638200
2021-01-07,119103.0,121983.0,119101.0,121956.0,121956.0,11774800
2021-01-08,122387.0,125324.0,122386.0,125077.0,125077.0,11085800
...,...,...,...,...,...,...
2021-12-23,105251.0,105453.0,104637.0,104891.0,104891.0,7283600
2021-12-27,104892.0,105694.0,104798.0,105531.0,105531.0,7594900
2021-12-28,105555.0,105652.0,104503.0,104864.0,104864.0,7509000


# **3. Explorando as funções da ta-lib e pandas_ta**

In [40]:
# A biblioteca pede para que crie um dataframe vazio antes de explorar as listas de indicadores disponíveis

df = pd.DataFrame()

In [50]:
# Lista dos indicadores disponíveis

df.ta.indicators()

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

In [51]:
# Função help para indicadores específicos

help(ta.bbands)

Help on function bbands in module pandas_ta.volatility.bbands:

bbands(close, length=None, std=None, ddof=0, mamode=None, talib=None, offset=None, **kwargs)
    Bollinger Bands (BBANDS)
    
    A popular volatility indicator by John Bollinger.
    
    Sources:
        https://www.tradingview.com/wiki/Bollinger_Bands_(BB)
    
    Calculation:
        Default Inputs:
            length=5, std=2, mamode="sma", ddof=0
        EMA = Exponential Moving Average
        SMA = Simple Moving Average
        STDEV = Standard Deviation
        stdev = STDEV(close, length, ddof)
        if "ema":
            MID = EMA(close, length)
        else:
            MID = SMA(close, length)
    
        LOWER = MID - std * stdev
        UPPER = MID + std * stdev
    
        BANDWIDTH = 100 * (UPPER - LOWER) / MID
        PERCENT = (close - LOWER) / (UPPER - LOWER)
    
    Args:
        close (pd.Series): Series of 'close's
        length (int): The short period. Default: 5
        std (int): The long 

In [58]:
# Obter a lista dos padrões de candles disponíveis na TA

cotacoes_ohlc.ta.cdl_pattern(name="all")

[X] Please install TA-Lib to use 2crows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3blackcrows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3inside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3linestrike. (pip install TA-Lib)
[X] Please install TA-Lib to use 3outside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3starsinsouth. (pip install TA-Lib)
[X] Please install TA-Lib to use 3whitesoldiers. (pip install TA-Lib)
[X] Please install TA-Lib to use abandonedbaby. (pip install TA-Lib)
[X] Please install TA-Lib to use advanceblock. (pip install TA-Lib)
[X] Please install TA-Lib to use belthold. (pip install TA-Lib)
[X] Please install TA-Lib to use breakaway. (pip install TA-Lib)
[X] Please install TA-Lib to use closingmarubozu. (pip install TA-Lib)
[X] Please install TA-Lib to use concealbabyswall. (pip install TA-Lib)
[X] Please install TA-Lib to use counterattack. (pip install TA-Lib)
[X] Please install TA-Lib to use darkcloudcover. (pip instal

,CDL_DOJI_10_0.1,CDL_INSIDE
Date,,
2021-01-04,0.0,0
2021-01-05,0.0,0
2021-01-06,0.0,0
2021-01-07,0.0,0
2021-01-08,0.0,0
...,...,...
2021-12-23,0.0,-1
2021-12-27,0.0,0
2021-12-28,0.0,0


# **4. Produzir figuras candles**

In [73]:
api_secret = 'trading_com_dados'
api_key = 'trading_com_dados'
client = Client(api_key,api_secret)
start_str = '2021-01-01'
end_str = '2022-01-01'

In [80]:
# Dados Bitcoin (BTCBRL)

btcbrl = client.get_historical_klines(symbol='BTCBRL', start_str=start_str, end_str=end_str, interval=Client.KLINE_INTERVAL_1DAY)
btcbrl_df = pd.DataFrame(btcbrl)
btcbrl_df.rename(columns={0:'Date',1:'Open',2:'High',3:'Low',4:'Close',5:'Volume'},inplace=True)
btcbrl_df.index = btcbrl_df['Date']
btcbrl_df.index = pd.to_datetime(btcbrl_df.index,unit='ms')
btcbrl_df = btcbrl_df.drop(btcbrl_df.columns[6:],axis=1)
btcbrl_df['Open'] = pd.to_numeric(btcbrl_df['Open'])
btcbrl_df['High'] = pd.to_numeric(btcbrl_df['High'])
btcbrl_df['Low'] = pd.to_numeric(btcbrl_df['Low'])
btcbrl_df['Close'] = pd.to_numeric(btcbrl_df['Close'])
btcbrl_df['Volume'] = pd.to_numeric(btcbrl_df['Volume'])
btcbrl_df

,Date,Open,High,Low,Close,Volume
Date,,,,,,
2021-01-01,1609459200000,152056.0,154167.0,150911.0,153480.0,58.948617
2021-01-02,1609545600000,153478.0,173900.0,152171.0,168784.0,187.818367
2021-01-03,1609632000000,168885.0,180925.0,167897.0,173503.0,172.847918
2021-01-04,1609718400000,173504.0,175981.0,150000.0,169820.0,245.309328
2021-01-05,1609804800000,169825.0,182966.0,159769.0,179972.0,229.906169
...,...,...,...,...,...,...
2021-12-28,1640649600000,288339.0,288339.0,269171.0,270226.0,198.005280
2021-12-29,1640736000000,270357.0,274941.0,264773.0,266975.0,175.657560
2021-12-30,1640822400000,266975.0,280000.0,263500.0,265750.0,245.152600


In [81]:
cotacoes_ohlc = btcbrl_df

In [119]:
data_ohlc = {'Candle_01': [2.5, 8, 2, 7.7], 'Candle_02': [6, 8, 2, 7.7],'Candle_03': [4, 8, 2, 6], 'Candle_04': [2, 8, 2, 2.7]}
df_figures = pd.DataFrame.from_dict(data_ohlc,orient='index',columns=['Open', 'High', 'Low', 'Close'])
df_figures

,Open,High,Low,Close
Candle_01,2.5,8,2,7.7
Candle_02,6.0,8,2,7.7
Candle_03,4.0,8,2,6.0
Candle_04,2.0,8,2,2.7


In [123]:
fig_cotacoes = go.Figure(data=[go.Candlestick(name="Ativo X",x=df_figures.index, open=df_figures['Open'], high = df_figures['High'], low=df_figures['Low'], close=df_figures['Close'])])
fig_cotacoes.update_layout(xaxis_rangeslider_visible=False, title_text='<b>Tipos candles vs força compradora',template = 'simple_white',width=500,height=500,showlegend=False)
fig_cotacoes.show()

In [ ]:
fig_cotacoes = go.Figure(data=[go.Candlestick(name="PETR3",x=cotacoes_df.index, open=cotacoes_df['open'], high = cotacoes_df['high'], low=cotacoes_df['low'], close=cotacoes_df['close'])])

fig_cotacoes.update_xaxes(title_text="<b> Data",
                          rangebreaks=[
    dict(bounds=["sat", "mon"]), # não mostrar finais de semana
    dict(bounds=[18, 10], pattern="hour"), # não mostrar horas de pregao fechado (observar cada ativo específico)
    dict(values=["2021-12-25","2022-01-01","2022-01-02"]) # não mostrar recesso de Ano Novo
    ])

fig_cotacoes.update_yaxes(title_text="<b> Preço Ativo (R$)")
fig_cotacoes.update_layout(xaxis_rangeslider_visible=False, title_text='Grafico Candlestick Ativo',template = 'simple_white',width=500,height=500,showlegend=True)
fig_cotacoes.show()

In [ ]:
sma_20p = ta.trend.SMAIndicator(ativos_ind_ITUB4['Adj Close'], window=20)
ativos_ind_ITUB4['SMA_20p'] = sma_20p.sma_indicator()

In [ ]:
ativos_ind_ITUB4.tail(5)

,Open,High,Low,Close,Adj Close,Volume,SMA_20p
Date,,,,,,,
2022-04-08,26.559999,26.900000,26.410000,26.660000,26.660000,26169900,26.840445
2022-04-11,26.600000,26.730000,26.440001,26.559999,26.559999,27257100,26.909754
2022-04-12,26.799999,26.900000,26.010000,26.090000,26.090000,29830700,26.960060
2022-04-13,26.280001,26.540001,25.969999,26.160000,26.160000,33866300,26.989381
2022-04-14,26.070000,26.469999,26.010000,26.330000,26.330000,23626400,26.999221


In [ ]:
ativos_ind_ITUB4['Meu_indicador_01'] = ativos_ind_ITUB4['Adj Close'] - ativos_ind_ITUB4['SMA_20p']
ativos_ind_ITUB4['Meu_indicador_02'] = ativos_ind_ITUB4['Close']/ativos_ind_ITUB4['High']

In [ ]:
ativos_ind_ITUB4

,Open,High,Low,Close,Adj Close,Volume,SMA_20p,Meu_indicador_01,Meu_indicador_02
Date,,,,,,,,,
2022-01-10,22.570000,22.879999,22.309999,22.840000,22.765636,41593900,NaN,NaN,0.998252
2022-01-11,22.790001,23.350000,22.700001,23.350000,23.273977,55923800,NaN,NaN,1.000000
2022-01-12,23.260000,23.500000,22.930000,23.250000,23.174303,41687000,NaN,NaN,0.989362
2022-01-13,23.090000,23.820000,23.049999,23.680000,23.602901,43414600,NaN,NaN,0.994123
2022-01-14,23.570000,23.850000,23.410000,23.500000,23.423489,29923100,NaN,NaN,0.985325
...,...,...,...,...,...,...,...,...,...
2022-04-08,26.559999,26.900000,26.410000,26.660000,26.660000,26169900,26.840445,-0.180445,0.991078
2022-04-11,26.600000,26.730000,26.440001,26.559999,26.559999,27257100,26.909754,-0.349754,0.993640
2022-04-12,26.799999,26.900000,26.010000,26.090000,26.090000,29830700,26.960060,-0.870059,0.969888


In [ ]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Candlestick(name='ITUB4', x=ativos_ind_ITUB4.index, open=ativos_ind_ITUB4['Open'], high = ativos_ind_ITUB4['High'], low=ativos_ind_ITUB4['Low'], close=ativos_ind_ITUB4['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=ativos_ind_ITUB4.index, y=ativos_ind_ITUB4['SMA_20p'],marker_color='blue'), row=1, col=1)

fig.add_trace(go.Scatter(name='Meu_indicador_01', x=ativos_ind_ITUB4.index, y=ativos_ind_ITUB4['Meu_indicador_01'],marker_color='green'), row=2, col=1)
fig.add_trace(go.Scatter(name='Meu_indicador_02', x=ativos_ind_ITUB4.index, y=ativos_ind_ITUB4['Meu_indicador_02'],marker_color='green'), row=3, col=1)


fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"])])
fig.update_layout(xaxis_rangeslider_visible=False, title_text='Cruzamento Médias Móveis',width=500,height=1000)
fig.show()